In [1]:
N = VER = 8
HOR = 1

def show(state, N=N):
    "Print a representation of a state as an NxN grid."
    # Initialize and fill in the board.
    board = [['.'] * N for _ in range(N)]
    for (c, squares) in state:
        for s in squares:
            x, y = divmod(s, N)
            board[x][y] = c
    # Now print it out
    for row in board:
        print(' '.join(row))

def direction(car):
	return HOR if car[0]+1 == car[1] else VER

def canmove(state, car, next_car):
	usedplaces = [n for a in state for n in a[1] if n not in car and a[0] != "@"]
	return True if len(set(next_car).intersection(set(usedplaces))) == 0 else False

def move(state, car, move):
	return tuple(map(sum, zip(car, move)))

def auth_pos(state, car, len_car):
	t = []
	mul = direction(car)
	for way in (-1, 1):
		next_car = move(state, car, tuple([way*mul]*len_car))
		while canmove(state, car, next_car):
			t.append(next_car)
			next_car = move(state, next_car, tuple([way*mul]*len_car))
	return t

def next_state(state, N=N):
	t = []
	for car in state:
		if car[0] != '@' and car[0] != '|':
			next_state = auth_pos(state, car[1], len(car[1]))
			if next_state:
				t.append(tuple((car[0],next_state)))
	return tuple(t)

def successors(state, N=N):
	res = {}
	new_state = next_state(state, N)
	for nstate in new_state:
		for n_pos in nstate[1]:
			res[tuple([c_state if c_state[0] != nstate[0] else (c_state[0], n_pos) for c_state in list(state)])] = tuple((nstate[0], n_pos))
	return res

# Here are the shortest_path_search and path_actions functions from the unit.
# You may use these if you want, but you don't have to.
def shortest_path_search(start, successors, is_goal):
    """Find the shortest path from start state to a state
    such that is_goal(state) is true."""
    if is_goal(start):
        return [start]
    explored = set() # set of states we have visited
    frontier = [ [start] ] # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop(0)
        s = path[-1]
        for (state, action) in successors(s).items():
            if state not in explored:
                explored.add(state)
                path2 = path + [action, state]
                if is_goal(state):
                    return path2
                else:
                    frontier.append(path2)
    return []
def path_actions(path):
    "Return a list of actions in this path."
    return path[1::2]

def bfs_solve_parking_puzzle(start, N=N):
    """Solve the puzzle described by the starting position (a tuple 
    of (object, locations) pairs).  Return a path of [state, action, ...]
    alternating items; an action is a pair (object, distance_moved),
    such as ('B', 16) to move 'B' two squares down on the N=8 grid."""
    def is_goal(state):
    	car_packed = [n for a in state for n in a[1]]
    	return len(set(car_packed))!=len(car_packed)

    # show the resolution
    sol = shortest_path_search(start, successors, is_goal)
    return sol[1::2]


# But it would also be nice to have a simpler format to describe puzzles,
# and a way to visualize states.
# You will do that by defining the following two functions:
def locs(start, n, incr=1):
    "Return a tuple of n locations, starting at start and incrementing by incr."
    return tuple(n for n in range(start, start + incr * n, incr))


def grid(cars, N=N):
    """Return a tuple of (object, locations) pairs -- the format expected for
    this puzzle.  This function includes a wall pair, ('|', (0, ...)) to 
    indicate there are walls all around the NxN grid, except at the goal 
    location, which is the middle of the right-hand wall; there is a goal
    pair, like ('@', (31,)), to indicate this. The variable 'cars'  is a
    tuple of pairs like ('*', (26, 27)). The return result is a big tuple
    of the 'cars' pairs along with the walls and goal pairs."""
    wall = [w for w in range(0, N + 1)] + \
           [w for w in range(2 * N - 1, N * (N - 1), N)] + \
           [w for w in range(2 * N, N * (N - 1), N)] + \
           [w for w in range(N * (N - 1), N * N)]
    issue = int((N * N - 1) / 2)
    wall.remove(issue)  # + tuple(('|', tuple(sorted(wall))))
    return tuple([('@', (issue,))] + [a for a in cars] + [('|', tuple(sorted(wall)))])


# Here we see the grid and locs functions in use:

puzzle1 = grid((
    ('*', locs(25, 2)),
    ('A', locs(11, 3, N)),
    ('B', locs(13, 2)),
    ('C', locs(22, 3, N)),
    ('D', locs(34, 2, N)),
    ('E', locs(35, 3)),
    ('F', locs(43, 2)),
    ('G', locs(45, 2, N))))


def depth_first_search(start, successors, is_goal):
    """Find the shortest path from start state to a state such that is_goal(state) is true,
    using depth-first search."""
    if is_goal(start):
        return [start]

    shortest_path = None  # Initialize shortest path to None
    shortest_length = float('inf')  # Initialize shortest path length to infinity

    explored = set()  # set of states we have visited
    frontier = [[start]]  # ordered list of paths we have blazed
    while frontier:
        path = frontier.pop()  # pop from the end of the list
        s = path[-1]

        if s not in explored:
            explored.add(s)
            for (state, action) in successors(s).items():
                if state not in explored:
                    new_path = path + [action, state]

                    # Check if a goal state is reached
                    if is_goal(state):
                        if len(new_path) < shortest_length:
                            shortest_path = new_path
                            shortest_length = len(new_path)
                    else:
                        frontier.append(new_path)  # append to the end of the list

    return shortest_path if shortest_path else []

def dfs_solve_parking_puzzle(start, N=N):
    """Solve the puzzle described by the starting position (a tuple 
    of (object, locations) pairs).  Return a path of [state, action, ...]
    alternating items; an action is a pair (object, distance_moved),
    such as ('B', 16) to move 'B' two squares down on the N=8 grid."""
    def is_goal(state):
        # il peut y avoir un overlap qu'en @ et si on overlap (si len differente) alors on est a la sortie
        car_packed = [n for a in state for n in a[1]]
        return len(set(car_packed))!=len(car_packed)

    # show the resolution
    sol = depth_first_search(start, successors, is_goal)
    return sol[1::2]

show(puzzle1)
print('Breadth First Search Path')
print(bfs_solve_parking_puzzle(puzzle1))
print('Total number of moves:', len(bfs_solve_parking_puzzle(puzzle1)))
print('\n')
print('Depth First Search Path')
print(dfs_solve_parking_puzzle(puzzle1))
print('Total number of moves:', len(dfs_solve_parking_puzzle(puzzle1)))

| | | | | | | |
| . . A . B B |
| . . A . . C |
| * * A . . C @
| . D E E E C |
| . D F F G . |
| . . . . G . |
| | | | | | | |
Breadth First Search Path
[('B', (12, 13)), ('C', (14, 22, 30)), ('D', (42, 50)), ('E', (34, 35, 36)), ('G', (21, 29)), ('E', (36, 37, 38)), ('F', (44, 45)), ('A', (35, 43, 51)), ('B', (9, 10)), ('G', (13, 21)), ('*', (28, 29)), ('A', (11, 19, 27)), ('E', (35, 36, 37)), ('C', (38, 46, 54)), ('*', (30, 31))]
Total number of moves: 15


Depth First Search Path
[('D', (42, 50)), ('E', (33, 34, 35)), ('G', (21, 29)), ('G', (37, 45)), ('G', (29, 37)), ('F', (45, 46)), ('G', (21, 29)), ('F', (44, 45)), ('G', (29, 37)), ('E', (34, 35, 36)), ('G', (21, 29)), ('F', (45, 46)), ('G', (29, 37)), ('F', (43, 44)), ('G', (45, 53)), ('G', (21, 29)), ('G', (37, 45)), ('C', (38, 46, 54)), ('G', (45, 53)), ('G', (21, 29)), ('G', (29, 37)), ('F', (44, 45)), ('G', (21, 29)), ('E', (35, 36, 37)), ('F', (43, 44)), ('E', (33, 34, 35)), ('G', (45, 53)), ('G', (29, 37)), ('G', (37, 45)